Read the JSON file as a Dataframe


In [8]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")



StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 10, Finished, Available)

In [9]:
display(df)

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, f74cead3-b242-4f2d-90d8-84e67221e654)

Explode the JSON column

In [10]:
from pyspark.sql.functions import explode
df_exploded=df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 12, Finished, Available)

In [11]:
display(df_exploded)

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9392b639-0b8c-4a5d-89d2-21899c5222e7)

Converting the Exploded JSON Dataframe to a single JSON string list

In [12]:
json_list=df_exploded.toJSON().collect()

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 14, Finished, Available)

Testing the JSON string list

In [13]:
print(json_list[2])

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 15, Finished, Available)

{"json_object":{"about":[{"name":"Academy Awards","readLink":"https://api.bing.microsoft.com/api/v7/entities/6b592a19-ed51-0b03-177b-890739d5ab35"},{"name":"Will Smith","readLink":"https://api.bing.microsoft.com/api/v7/entities/ee4ac44e-f1b1-103d-1a41-6b281462b58c"},{"name":"The Fresh Prince of Bel-Air","readLink":"https://api.bing.microsoft.com/api/v7/entities/9bfeff35-b09d-5dff-5239-bf2210807fa3"},{"name":"Where the Day Takes You","readLink":"https://api.bing.microsoft.com/api/v7/entities/f7c8807f-91a7-9ac6-26a4-54b01f548e59"}],"category":"Entertainment","datePublished":"2024-06-05T18:40:00.0000000Z","description":"Smith began his career in television, breaking out in The Fresh Prince of Bel-Air (1990–1996), and later found his way into the world of films with the 1992 drama Where the Day Takes You. From there, he’s gathered an impressive filmography with massive blockbusters, indie darlings, and the occasional box office flop.","image":{"thumbnail":{"contentUrl":"https://www.bing.co

In [15]:
import json
news_json=json.loads(json_list[2])

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 17, Finished, Available)

In [16]:
print(news_json)

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 18, Finished, Available)

{'json_object': {'about': [{'name': 'Academy Awards', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/6b592a19-ed51-0b03-177b-890739d5ab35'}, {'name': 'Will Smith', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/ee4ac44e-f1b1-103d-1a41-6b281462b58c'}, {'name': 'The Fresh Prince of Bel-Air', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/9bfeff35-b09d-5dff-5239-bf2210807fa3'}, {'name': 'Where the Day Takes You', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/f7c8807f-91a7-9ac6-26a4-54b01f548e59'}], 'category': 'Entertainment', 'datePublished': '2024-06-05T18:40:00.0000000Z', 'description': 'Smith began his career in television, breaking out in The Fresh Prince of Bel-Air (1990–1996), and later found his way into the world of films with the 1992 drama Where the Day Takes You. From there, he’s gathered an impressive filmography with massive blockbusters, indie darlings, and the occasional box office flop.', 'image': {'thumbnail': {'conten

In [17]:
# Sample news_json object
news_json = {
    'json_object': {
        "name": "T20 World Cup: India v Ireland - clips, radio & text",
        "description": "India face Ireland in their opening game of the Men's T20 World Cup 2024 in New York - follow text, radio commentary and video highlights.",
        "category": "Sports",
        "url": "https://www.bbc.com/sport/cricket/live/c5119q15ygdt",
        # "image": {  # Uncomment this block to simulate the presence of the image key
        #     "thumbnail": {
        #         "contentUrl": "https://example.com/image.jpg"
        #     }
        # },
        "provider": [
            {"name": "BBC Sport"}
        ],
        "datePublished": "2024-06-05T12:34:56Z"
    }
}

# Safely access each field with default values
name = news_json['json_object'].get("name", "N/A")
description = news_json['json_object'].get("description", "N/A")
category = news_json['json_object'].get("category", "N/A")
url = news_json['json_object'].get("url", "N/A")
image_url = news_json['json_object'].get("image", {}).get("thumbnail", {}).get("contentUrl", "No image available")
provider_name = news_json['json_object'].get("provider", [{}])[0].get('name', "N/A")
date_published = news_json['json_object'].get("datePublished", "N/A")

print(name)
print(description)
print(category)
print(url)
print(image_url)
print(provider_name)
print(date_published)


StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 19, Finished, Available)

T20 World Cup: India v Ireland - clips, radio & text
India face Ireland in their opening game of the Men's T20 World Cup 2024 in New York - follow text, radio commentary and video highlights.
Sports
https://www.bbc.com/sport/cricket/live/c5119q15ygdt
No image available
BBC Sport
2024-06-05T12:34:56Z


Processing the JSON property to list

In [18]:
title = []
description = []
category = []
url = []
image = []
provider = []
datePublished = []
# Process each JSON object in the list
for json_str in json_list:
    try:
# Parse the JSON string into a dictionary
        article = json. loads(json_str)
        if article["json_object"].get("category")and article["json_object"].get("image",{}).get("thumbnail",{}).get("contentUrl"):
#Extract information from the dictionary
            title.append (article["json_object"] ["name"] )
            description.append(article["json_object"] ["description"] )
            category.append (article["json_object"] ["category"])
            url.append (article["json_object"] ["url"])
            image.append (article["json_object"] ["image"] ["thumbnail"] ["contentUrl"])
            provider.append (article["json_object"] ["provider"] [0] ['name'])
            datePublished.append (article["json_object"] ["datePublished"])
    except Exception as e:
      print(f"Error processing JSON object: {e}")

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 20, Finished, Available)

Converting the List to a Dataframe

In [19]:
from pyspark.sql.types import StructType, StructField, StringType
# Combine the lists
data = list(zip(title, description, category,url, image, provider,datePublished) )
# Define schema
schema = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
    ])
# Create DataFrame
df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 21, Finished, Available)

In [20]:
display(df_cleaned)

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, f086205d-3fb5-45ae-b108-14ee927a411c)

Processing the Date column

In [21]:
from pyspark.sql.functions import to_date,date_format
df_cleaned_final=df_cleaned.withColumn("datePublished",date_format(to_date("datePublished"),"dd-MMM-yyyy"))

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 23, Finished, Available)

In [22]:
display(df_cleaned_final.limit(5))

StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8ff1b2e7-c5b7-46a1-8cf9-afef406f30b7)

In [23]:
from pyspark.sql.utils import AnalysisException

table_name = 'bing_lake_db.tbl_latest_news'

# Function to check if the table exists
def table_exists(table_name):
    try:
        spark.sql(f"DESCRIBE TABLE {table_name}")
        return True
    except AnalysisException:
        return False

# Check if the table exists
if not table_exists(table_name):
    # If the table doesn't exist, create it
    df_cleaned_final.write.format("delta").saveAsTable(table_name)
else:
    # If the table exists, perform the merge operation
    print("Table Already Exists")
    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_df_cleaned_final AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR 
            source_view.category <> target_table.category OR 
            source_view.image <> target_table.image OR 
            source_view.provider <> target_table.provider OR 
            source_view.datePublished <> target_table.datePublished
        )
        THEN UPDATE SET
            target_table.title = source_view.title,
            target_table.description = source_view.description,
            target_table.category = source_view.category,
            target_table.image = source_view.image,
            target_table.provider = source_view.provider,
            target_table.datePublished = source_view.datePublished
        WHEN NOT MATCHED THEN
        INSERT *
    """)


StatementMeta(, 9b51587a-a89e-4bdd-8bcd-e70e346961c3, 25, Finished, Available)

Table Already Exists
